In [70]:
import numpy as np
import pandas as pd
from io import StringIO
from geo_utils import calculate_dist

In [97]:
def read_alert_data(filename, include_region=True):
    rd = open(filename, "r")
    out = []
    while True:
        # Read next line
        line = rd.readline()
        # If line is blank, then you struck the EOF
        if len(line) > 0:
            if line[:2] == 'E:':
                out.append(line.strip())

        if not line :
            break;
    if include_region:
        df = pd.read_csv(StringIO('\n'.join(out)),
                 delim_whitespace=True, 
                 names=['E', 'id', 'evid', 'mag', 'evla', 'evlo',
                          'date', 'time', 'system', 'alert_time', 'est_mag',
                          'est_evla', 'est_evlo', 'est_date', 'est_time', 'region'])
    else:
        df = pd.read_csv(StringIO('\n'.join(out)),
                 delim_whitespace=True, 
                 names=['E', 'id', 'evid', 'mag', 'evla', 'evlo',
                          'date', 'time', 'system', 'alert_time', 'est_mag',
                          'est_evla', 'est_evlo', 'est_date', 'est_time'])
    
    return df

In [98]:
# Note, the b indicate the 2nd version Ivan gave me

filename = '../data/shakeAlert/alerts2016c.txt'
df_alert_2016 = read_alert_data(filename, include_region=True)
filename = '../data/shakeAlert/alerts2018c.txt'
df_alert_2018 = read_alert_data(filename, include_region=True)

In [99]:
df_alert = pd.concat([df_alert_2016, df_alert_2018])

In [91]:
df_eq = pd.read_pickle('../data/USGS_Earthquake_20160201_20200801_M3.5above.pkl')

# Get metrics
In this section, we will get the True positive, False positive, and False negative in terms of detection. 

* True positive - SA sent out alert for the USGS event. 
* False positive - SA sent out alert but no corresponding USGS event. 
* False negative - SA didn't send out an alert for USGS event. 

### Get the True positive and False negative

In [100]:
false_negative_list = []
true_positive_list = []

for ix, row in df_eq.iterrows():
    evid = row['id']
    
    if evid in df_alert.evid.to_list():
        true_positive_list.append(evid)
    else:
        false_negative_list.append(evid)

In [101]:
len(true_positive_list)

465

In [94]:
len(false_negative_list)

311

### Get the False positive

In [133]:
df_eq_smaller = pd.read_pickle('../data/USGS_Earthquake_20160201_20200801_M2.5_M3.5.pkl')

In [134]:
true_positive_alert_ids = [item[1] for item in true_positive_list]

In [152]:
df_matched_events = pd.read_csv('../data/matched_event.csv')
df_matched_events_2016 = pd.read_csv('../data/matched_event_2016.csv')

In [166]:
data_list = []
for _, row in df_matched_events.iterrows():
    data_list.append([row['id'], row['ver'],row['catid'],
                      row['time'], row['day'], row['dsecs'],
                      row['dkm'], row['dmag'], row['modtime'],
                      row['id.1'], row['catalogid'], row['lat'],
                      row['lon'], row['depth'], row['mag'],
                      row['time.1'], row['day.1'], row['modtime.1']])

In [169]:
for _, row in df_matched_events_2016.iterrows():
    data_list.append([row['id'], row['ver'],row['catid'],
                      row['time'], row['day'], row['dsecs'],
                      row['dkm'], row['dmag'], row['modtime'],
                      row['id.1'], row['catalog'], row['lat'],
                      row['lon'], row['depth'], row['mag'],
                      row['time.1'], row['day.1'], row['modtime.1']])

In [173]:
df_matched_events = pd.DataFrame(data_list, columns=['id', 'ver', 'catid', 'time', 'day', 'dsecs', 'dkm', 'dmag', 'modtime',
       'id.1', 'catalogid', 'lat', 'lon', 'depth', 'mag', 'time.1', 'day.1',
       'modtime.1'])

df_matched_events = df_matched_events.sort_values('time')
df_matched_events.to_csv('matched_events_from_SA_20160201_20201006.csv')

In [178]:
len(true_positive_list)

508

In [179]:
len(false_negative_list)

280